In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install rich

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 237 kB 24.7 MB/s 
     |████████████████████████████████| 51 kB 7.7 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import os
import re
import rich  # 출력을 예쁘게 꾸며주는 라이브러리
from rich.table import Table

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier
from tqdm.auto import tqdm

from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

train_df = pd.read_csv('/content/gdrive/My Drive/kerc/data/tmp/df_train_extended.tsv',sep = '\t')
train_labels_df = pd.read_csv('/content/gdrive/My Drive/kerc/data/train_labels.csv')
test_df = pd.read_csv('/content/gdrive/My Drive/kerc/data/tmp/df_public_test_extended.tsv', sep = '\t')
#submission = pd.read_csv('/content/gdrive/My Drive/kerc/data/sample_submission.csv')




In [ ]:
train_labels_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7339 entries, 0 to 7338
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sentence_id  7339 non-null   int64 
 1   label        7339 non-null   object
dtypes: int64(1), object(1)
memory usage: 114.8+ KB


In [ ]:
train_df = pd.merge(train_df, train_labels_df, on = 'sentence_id', how = 'left')

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7339 entries, 0 to 7338
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   sentence_id         7339 non-null   int64 
 1   person              7339 non-null   object
 2   sentence            7339 non-null   object
 3   scene               7339 non-null   object
 4   context             5322 non-null   object
 5   target_speaker_ctx  7339 non-null   object
 6   other_speaker_ctx   7339 non-null   object
 7   scene_sents         7339 non-null   object
 8   label               7339 non-null   object
dtypes: int64(1), object(8)
memory usage: 573.4+ KB


In [ ]:
train_df.head()

,sentence_id,person,sentence,scene,context,target_speaker_ctx,other_speaker_ctx,scene_sents,label
0,1,어영,야! 전화 받아. 아무리 바빠도 내전화는 받아야 되는거 아냐? 약속 하나도 못지키는...,S0104,NaN,,,,dysphoria
1,2,어영,우리 아무래도 안되겠다. 이게 최선인거 같애. 평생 잊지 않을게. 행복하길 바란다.,S0108,"포기한듯 탁 일어서는데, 띵동 문자. 후다닥 보는 어영. 기막혀 읽어보는",,,,dysphoria
2,3,지구대,"김경사님, 아직 안가셨어요? 시간 다됐을텐데.",S0109,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대,,,,neutral
3,4,순경,근무중인데 어딜가?,S0109,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대,,"김경사님, 아직 안가셨어요? 시간 다됐을텐데.","김경사님, 아직 안가셨어요? 시간 다됐을텐데.",dysphoria
4,5,지구대,다녀오세요. 이런날은 무조건 가서 축하해주셔야죠. 이순경이 대신 근무선다고 나온대요.,S0109,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대,"김경사님, 아직 안가셨어요? 시간 다됐을텐데.",근무중인데 어딜가?,"김경사님, 아직 안가셨어요? 시간 다됐을텐데.근무중인데 어딜가?",euphoria


In [ ]:
train_df = train_df.replace(np.nan, '', regex=True)

In [ ]:
test_df = test_df.replace(np.nan, '', regex=True)

In [ ]:
train_df.head()

,sentence_id,person,sentence,scene,context,target_speaker_ctx,other_speaker_ctx,scene_sents,label
0,1,어영,야! 전화 받아. 아무리 바빠도 내전화는 받아야 되는거 아냐? 약속 하나도 못지키는...,S0104,,,,,dysphoria
1,2,어영,우리 아무래도 안되겠다. 이게 최선인거 같애. 평생 잊지 않을게. 행복하길 바란다.,S0108,"포기한듯 탁 일어서는데, 띵동 문자. 후다닥 보는 어영. 기막혀 읽어보는",,,,dysphoria
2,3,지구대,"김경사님, 아직 안가셨어요? 시간 다됐을텐데.",S0109,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대,,,,neutral
3,4,순경,근무중인데 어딜가?,S0109,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대,,"김경사님, 아직 안가셨어요? 시간 다됐을텐데.","김경사님, 아직 안가셨어요? 시간 다됐을텐데.",dysphoria
4,5,지구대,다녀오세요. 이런날은 무조건 가서 축하해주셔야죠. 이순경이 대신 근무선다고 나온대요.,S0109,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대,"김경사님, 아직 안가셨어요? 시간 다됐을텐데.",근무중인데 어딜가?,"김경사님, 아직 안가셨어요? 시간 다됐을텐데.근무중인데 어딜가?",euphoria


In [ ]:
test_df.head()

,sentence_id,person,sentence,scene,context,target_speaker_ctx,other_speaker_ctx,scene_sents
0,10,현찰,제가 집안일땜에 잠깐 나와 있거든요. 시간안에 가겠습니다.,S0113,"음악소리. 경찰의날, 행사 단상차려진. 정복 예복차려입은 경찰들 앉았고. 일반 사람...",,,
1,11,현찰,"계약할 서류는 태연희실장 통해서 보내 드렸는데 확인해 보셨죠? 예, 예.",S0113,"음악소리. 경찰의날, 행사 단상차려진. 정복 예복차려입은 경찰들 앉았고. 일반 사람...",제가 집안일땜에 잠깐 나와 있거든요. 시간안에 가겠습니다.,,제가 집안일땜에 잠깐 나와 있거든요. 시간안에 가겠습니다.
2,12,경위1,김순경 경사님 사모님 아니십니까?,S0113,"음악소리. 경찰의날, 행사 단상차려진. 정복 예복차려입은 경찰들 앉았고. 일반 사람...",,제가 집안일땜에 잠깐 나와 있거든요. 시간안에 가겠습니다. 계약할 서류는 태연희실...,제가 집안일땜에 잠깐 나와 있거든요. 시간안에 가겠습니다.계약할 서류는 태연희실장...
3,13,과자,아 예.,S0113,"음악소리. 경찰의날, 행사 단상차려진. 정복 예복차려입은 경찰들 앉았고. 일반 사람...",,제가 집안일땜에 잠깐 나와 있거든요. 시간안에 가겠습니다. 계약할 서류는 태연희실...,제가 집안일땜에 잠깐 나와 있거든요. 시간안에 가겠습니다.계약할 서류는 태연희실장...
4,14,경위1,저 기억하시죠? 00지구대에서 김경사님 모시고 근무했었습니다.,S0113,"음악소리. 경찰의날, 행사 단상차려진. 정복 예복차려입은 경찰들 앉았고. 일반 사람...",김순경 경사님 사모님 아니십니까?,제가 집안일땜에 잠깐 나와 있거든요. 시간안에 가겠습니다. 계약할 서류는 태연희실...,제가 집안일땜에 잠깐 나와 있거든요. 시간안에 가겠습니다.계약할 서류는 태연희실장...


In [ ]:
# 두 칸 이상의 빈칸을 한칸으로 바꿔줌
train_df["sentence"]  = train_df["sentence"].str.replace("\s+", " ", regex=True)+" "+train_df["context"].str.replace("\s+", " ", regex=True)+" "+train_df["target_speaker_ctx"].str.replace("\s+", " ", regex=True)+" "+train_df["other_speaker_ctx"].str.replace("\s+", " ", regex=True)+" "+train_df["scene_sents"].str.replace("\s+", " ", regex=True)

In [ ]:
test_df["sentence"]  = test_df["sentence"].str.replace("\s+", " ", regex=True)+" "+test_df["context"].str.replace("\s+", " ", regex=True)+" "+test_df["target_speaker_ctx"].str.replace("\s+", " ", regex=True)+" "+test_df["other_speaker_ctx"].str.replace("\s+", " ", regex=True)+" "+test_df["scene_sents"].str.replace("\s+", " ", regex=True)

In [ ]:
def preprocess(text):
  text = text.lower()
  text = re.sub("[^A-Za-z가-힣 ]","", text)  # re.sub(r"[^A-Za-zㄱ-ㅎㅏ-ㅣ가-힣 ]","", text)
  return text

train_df['preprocessed'] = train_df.sentence.apply(lambda x : preprocess(x))
test_df['preprocessed'] = test_df.sentence.apply(lambda x : preprocess(x))

train_df.head()

,sentence_id,person,sentence,scene,context,target_speaker_ctx,other_speaker_ctx,scene_sents,label,preprocessed
0,1,어영,야! 전화 받아. 아무리 바빠도 내전화는 받아야 되는거 아냐? 약속 하나도 못지키는...,S0104,,,,,dysphoria,야 전화 받아 아무리 바빠도 내전화는 받아야 되는거 아냐 약속 하나도 못지키는 주제...
1,2,어영,우리 아무래도 안되겠다. 이게 최선인거 같애. 평생 잊지 않을게. 행복하길 바란다....,S0108,"포기한듯 탁 일어서는데, 띵동 문자. 후다닥 보는 어영. 기막혀 읽어보는",,,,dysphoria,우리 아무래도 안되겠다 이게 최선인거 같애 평생 잊지 않을게 행복하길 바란다 포기한...
2,3,지구대,"김경사님, 아직 안가셨어요? 시간 다됐을텐데. 초조하게 시계보면서 왔다갔다 서성이는...",S0109,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대,,,,neutral,김경사님 아직 안가셨어요 시간 다됐을텐데 초조하게 시계보면서 왔다갔다 서성이는 김순...
3,4,순경,근무중인데 어딜가? 초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서...,S0109,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대,,"김경사님, 아직 안가셨어요? 시간 다됐을텐데.","김경사님, 아직 안가셨어요? 시간 다됐을텐데.",dysphoria,근무중인데 어딜가 초조하게 시계보면서 왔다갔다 서성이는 김순경 순찰차경차와서 멈추고...
4,5,지구대,다녀오세요. 이런날은 무조건 가서 축하해주셔야죠. 이순경이 대신 근무선다고 나온대요...,S0109,초조하게 시계보면서 왔다갔다 서성이는 김순경. 순찰차(경차)와서 멈추고 내리는 지구대,"김경사님, 아직 안가셨어요? 시간 다됐을텐데.",근무중인데 어딜가?,"김경사님, 아직 안가셨어요? 시간 다됐을텐데.근무중인데 어딜가?",euphoria,다녀오세요 이런날은 무조건 가서 축하해주셔야죠 이순경이 대신 근무선다고 나온대요 초...


In [ ]:
def get_pipe(model, model_name: str) -> Pipeline:
    "TfidfVectorizer와 모델을 연결한 파이프라인을 반환하는 함수"
    tfidf = TfidfVectorizer(analyzer="char", ngram_range=(1, 3))
    pipe = Pipeline([
        ("tfidf", tfidf),
        (model_name, model)
    ])
    return pipe

In [ ]:
def return_kfold_accuarcy(model, k: int = 5) -> float:
    "모델을 입력받아 KFold 예측 후 accuracy score를 반환하는 함수"
    kfold = StratifiedKFold(k, shuffle=True, random_state=42)
    result = []
    for train_idx, test_idx in kfold.split(train_df["sentence"], train_df["label"]):
        train, val = train_df.iloc[train_idx], train_df.iloc[test_idx]
        model.fit(train["sentence"], train["label"])
        pred = model.predict(val["sentence"])
        acc = accuracy_score(val["label"], pred)
        result.append(acc)

    return np.mean(result)

In [ ]:
models = [
    ("naive_bayes", BernoulliNB()),
    ("SGD", SGDClassifier(random_state=42, n_jobs=-1)),
    # ("rfc", RandomForestClassifier(random_state=42, n_jobs=-1)),
    # ("SVC", SVC(random_state=42)),
    # ("ada", AdaBoostClassifier(random_state=42)),
    # ("lgbm", LGBMClassifier(random_state=42)),
    ("lgbm2", LGBMClassifier(n_estimators=3, random_state=42)),
    # ("xgb", XGBClassifier(random_state=42)),
    # ("knc1", KNeighborsClassifier()),
    # ("knc2", KNeighborsClassifier(n_neighbors=4))
]

model_pipes = [(name, get_pipe(model, name)) for name, model in models]

In [ ]:
table = Table(title="Model Comparison Table")
table.add_column("Model Name", justify="left", style="green")
table.add_column("Accuracy", justify="right")

for model_name, model in tqdm(model_pipes, leave=False):
    acc = return_kfold_accuarcy(model)
    table.add_row(model_name, f"{acc:0.3f}")

rich.print(table)

  0%|          | 0/3 [00:00<?, ?it/s]

  Model Comparison Table  
┏━━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Model Name  ┃ Accuracy ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━┩
│ naive_bayes │    0.605 │
│ SGD         │    0.819 │
│ lgbm2       │    0.617 │
└─────────────┴──────────┘

In [ ]:
from sklearn.ensemble import StackingClassifier

stack_models = [(name, get_pipe(model, name)) for name, model in models]

stacking = StackingClassifier(stack_models)
acc = return_kfold_accuarcy(stacking)
rich.print(acc)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


0.7874375219404557

In [ ]:
stacking.fit(train_df['sentence'], train_df['label'])
submission_pred = stacking.predict(test_df['sentence'])

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
submission_pred

array(['dysphoria', 'dysphoria', 'dysphoria', ..., 'dysphoria',
       'euphoria', 'euphoria'], dtype=object)

In [ ]:
submission_df = pd.DataFrame()
submission_df["Id"] = test_df['sentence_id']
submission_df["Predicted"] = submission_pred
submission_df.to_csv(f'/content/gdrive/MyDrive/kerc/logs/submission_TEAM_NAME.csv', index=False)
print(f"Saved 'logs/submission_TEAM_NAME.csv")

Saved 'logs/submission_TEAM_NAME.csv
